In [1]:
from model import atp_graph
from data import synthetic_data_gen, feature_extractor
import keras
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import numpy as np
import tensorflow as tf
import IPython
import time
import os

from model import atp_pipeline

2023-04-28 16:42:52.853765: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import pandas as pd

In [3]:
pd_array = pd.read_csv("datasets/weather.csv")

In [4]:
data = np.array(pd_array)

In [5]:
data.shape

(52696, 22)

In [6]:
data[:,0] = np.linspace(-1,1,data.shape[0])

In [7]:
data = data.astype("float32")

In [8]:
training_data = data[:int(0.7*data.shape[0])]
val_data = data[int(0.7*data.shape[0]):int(0.8*data.shape[0])]
test_data = data[int(0.9*data.shape[0]):]

In [9]:
#scale

training_data_scaled = (training_data - np.mean(training_data,axis=0))/np.std(training_data,axis=0)
val_data_scaled = (val_data - np.mean(training_data,axis=0))/np.std(training_data,axis=0)
test_data_scaled =  (test_data - np.mean(training_data,axis=0))/np.std(training_data,axis=0)

In [10]:
data.shape

(52696, 22)

In [11]:
save_dir = "/users/rapart57/rapart57-hpc/ATP/weights/forecasting/weather"

In [12]:
tf.random.set_seed(442)
opt = tf.keras.optimizers.Adam(3e-4)
run=2; heads=2
atp_model = atp_pipeline.atp_pipeline()

EPOCHS = 2



2023-04-28 16:42:56.291912: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-28 16:42:56.889164: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6975 MB memory:  -> device: 0, name: Quadro P4000, pci bus id: 0000:3b:00.0, compute capability: 6.1


In [36]:
def batcher(t, y, batch_s = 32,given_hr = 96, pred_hr = 192):
    win_l = given_hr + pred_hr
    

    idx = np.random.choice(np.arange(0, len(y) - win_l), batch_s, replace=False)
    
    y = np.array([np.array(y)[i:i+win_l] for i in idx])
    t = np.array([np.array(t)[i:i+win_l] for i in idx])
    
    return t, y


In [42]:
a,b = create_batch([val_data_scaled[:,:1],val_data_scaled[:,-2:]],batch_s=val_data_scaled.shape[0])

In [44]:
b.shape


(5269, 2)

In [28]:
val_data_scaled.shape[0] 

5269

In [35]:
t_tr,y_tr = batcher(val_data_scaled[:,:1],val_data_scaled[:,-2:],batch_s = val_data_scaled.shape[0] -seq_l)


[ 288  289  290 ... 4978 4979 4980]


In [14]:
seq_l = 96+192;
num_batches = 500
batch_s=20
mini = 50000
n_C_te =5

In [15]:
x,y = batcher(training_data_scaled[:,:-1],training_data_scaled[:,-1:])

In [16]:
x.shape

(32, 288, 21)

In [17]:
y.shape

(32, 288, 1)

In [18]:
step = 1
tr_step, te_step = atp_graph.build_graph()

name_comp = 'run_' + str(run)
logdir = save_dir + '/logs/' + name_comp
folder = save_dir + '/ckpt/check_' + name_comp

if not os.path.exists(folder): os.mkdir(folder)
ckpt = tf.train.Checkpoint(step=tf.Variable(1), optimizer=opt, net=atp_model)
manager = tf.train.CheckpointManager(ckpt, folder, max_to_keep=1)
ckpt.restore(manager.latest_checkpoint)

In [38]:
for epoch in range(2):
    print("epoch",+epoch)
    start = time.time()
    for batch_n in range(10):
        
        
        x,y = batcher(training_data_scaled[:,:1],training_data_scaled[:,-2:])
       
        n_C = int(np.random.choice(np.arange(2, 20), 1))
        n_T = 288 - n_C
        y_tr = y
        t_tr = x

        _,_, nll_pp, msex = tr_step(atp_model, opt, t_tr,y_tr,n_C,n_T)

        if ((batch_n % 2 == 0)):
            
            
            t_tr,y_tr = batcher(val_data_scaled[:,:1],val_data_scaled[:,-2:],batch_s = val_data_scaled.shape[0]-288)
            
            _,_, nll_pp_te, msex_te = te_step(atp_model, t_tr,y_tr,10,200)

        if nll_pp_te < mini:
            mini = nll_pp_te
            IPython.display.clear_output(wait=True)
            print("epoch {} batch {} test lik pp: {}".format(epoch, batch_n, nll_pp)) 
            manager.save()
            step += 1
            ckpt.step.assign_add(1)
    print ('Time taken for 1 epoch: {} secs\n'.format(time.time() - start))

epoch 0
(32, 288, 2)
(4981, 210, 2)


2023-04-28 16:49:25.947021: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.56GiB (rounded to 1673616640)requested by op atp_pipeline/de/argsort_1/TopKV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-28 16:49:25.947110: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2023-04-28 16:49:25.947161: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 307, Chunks in use: 307. 76.8KiB allocated for chunks. 76.8KiB in use in bin. 39.7KiB client-requested in use in bin.
2023-04-28 16:49:25.947186: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2023-04-28 16:49:25.947213: I tensorflow/ts

ResourceExhaustedError: Graph execution error:

Detected at node 'atp_pipeline/de/argsort_1/TopKV2' defined at (most recent call last):
    File "/data/hpcdata/users/rapart57/conda-envs/updated_tf/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/data/hpcdata/users/rapart57/conda-envs/updated_tf/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/data/hpcdata/users/rapart57/conda-envs/updated_tf/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/data/hpcdata/users/rapart57/conda-envs/updated_tf/lib/python3.10/site-packages/traitlets/config/application.py", line 1043, in launch_instance
      app.start()
    File "/data/hpcdata/users/rapart57/conda-envs/updated_tf/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/data/hpcdata/users/rapart57/conda-envs/update